### Launch Jobs for training

In [1]:
import os
import ads

from ads.jobs import DataScienceJob
from ads.jobs import DataScienceJobRun
from ads.jobs import ScriptRuntime
from ads.jobs import Job

from ads import set_auth

In [2]:
# compartment_id = os.environ['NB_SESSION_COMPARTMENT_OCID']
# project_id = os.environ['PROJECT_OCID']

set_auth(auth='resource_principal')

In [3]:
#
# Here all the configurations for the JOB
#
LOG_GROUP_ID = "ocid1.loggroup.oc1.eu-frankfurt-1.amaaaaaangencdyazs4l4rzrzsarlej6mqlwlbz6bmnx4adwdlssveam2jaa"
LOG_ID = "ocid1.log.oc1.eu-frankfurt-1.amaaaaaangencdya47httqmxyiew5tkxa6l7gekev2ljpasixuhmp2fa3v5q"

NAMESPACE = "frqap2zhtzbe"
# the bucket with the custom CONDA env
CONDA_BUCKET = "whisper_jobs_env"
# bucket with code to execute
SOURCE_BUCKET = "whisper_jobs"

CUSTOM_ENV_URI = f"oci://{CONDA_BUCKET}@{NAMESPACE}/conda_environments/gpu/whisper_env_/1.0/whisper_env_v1_0"
SOURCE_URI = f"oci://{SOURCE_BUCKET}@{NAMESPACE}/test_atco2.tar.gz"

# the first file to execute
RUN_ENTRYPOINT = "train.sh"

# SHAPE_NAME = "VM.Standard2.4"
# SHAPE_NAME = "VM.GPU2.1"
SHAPE_NAME = "VM.GPU.A10.2"
# in GB
STORAGE_SIZE = 2000

JOBS_NAME = "job_atco2_007"

In [4]:
# 1. Specify the Infrastructure requested
# VM Shape, logging
# network is taken from NB session

# you need to provide the OCID for LogGroup and Log
infrastructure = (
    DataScienceJob()
    .with_shape_name(SHAPE_NAME)
    .with_block_storage_size(STORAGE_SIZE)
    .with_log_group_id(LOG_GROUP_ID)
    .with_log_id(LOG_ID)
)

In [5]:
# specify the runtime and conda and env 
runtime = (
    ScriptRuntime()
    .with_source(SOURCE_URI)
    .with_custom_conda(CUSTOM_ENV_URI)
    .with_environment_variable(JOB_RUN_ENTRYPOINT=RUN_ENTRYPOINT)
)

In [6]:
# specify the JOB
job = (
    Job(name=JOBS_NAME)
    .with_infrastructure(infrastructure)
    .with_runtime(runtime)
)

In [7]:
# create the JOB
job.create()

kind: job
spec:
  id: ocid1.datasciencejob.oc1.eu-frankfurt-1.amaaaaaangencdyawfbeauoxq3to4c7iweu33dq2jrc5h2thle6jmfpjvgfq
  infrastructure:
    kind: infrastructure
    spec:
      blockStorageSize: 2000
      compartmentId: ocid1.compartment.oc1..aaaaaaaag2cpni5qj6li5ny6ehuahhepbpveopobooayqfeudqygdtfe6h3a
      displayName: job_atco2_007
      jobInfrastructureType: STANDALONE
      jobType: DEFAULT
      logGroupId: ocid1.loggroup.oc1.eu-frankfurt-1.amaaaaaangencdyazs4l4rzrzsarlej6mqlwlbz6bmnx4adwdlssveam2jaa
      logId: ocid1.log.oc1.eu-frankfurt-1.amaaaaaangencdya47httqmxyiew5tkxa6l7gekev2ljpasixuhmp2fa3v5q
      projectId: ocid1.datascienceproject.oc1.eu-frankfurt-1.amaaaaaangencdyasybymsgwfmwo7ukyjs6kdl573kpxnb5rgy52c5irb5pq
      shapeName: VM.GPU.A10.2
      subnetId: ocid1.subnet.oc1.eu-frankfurt-1.aaaaaaaaijgqblnhpqle2zorl75qli23wre5eboqjtystagdgun4qwdxj4aq
    type: dataScienceJob
  name: job_atco2_007
  runtime:
    kind: runtime
    spec:
      conda:
        type: publ

In [8]:
# get the job run by id
# JOB_RUN_OCID = "ocid1.datasciencejobrun.oc1.eu-frankfurt-1.amaaaaaangencdyais5w2gkzbqbwqeftps6raay65ymhg46hltw3vlokij3q"

# job_run = DataScienceJobRun.from_ocid(JOB_RUN_OCID)

In [9]:
job_run = job.run()

In [ ]:
job_run.watch()

Job OCID: ocid1.datasciencejob.oc1.eu-frankfurt-1.amaaaaaangencdyawfbeauoxq3to4c7iweu33dq2jrc5h2thle6jmfpjvgfq
Job Run OCID: ocid1.datasciencejobrun.oc1.eu-frankfurt-1.amaaaaaangencdyasd7tjtnzu5ennlv3mogdzvmvij52fqvz7lhfbur7m4va
2023-09-11 09:16:27 - Job Run ACCEPTED
2023-09-11 09:16:43 - Job Run ACCEPTED, Infrastructure provisioning.
2023-09-11 09:17:55 - Job Run ACCEPTED, Infrastructure provisioned.
2023-09-11 09:18:29 - Job Run ACCEPTED, Job run bootstrap starting.
2023-09-11 09:21:58 - Job Run ACCEPTED, Job run bootstrap complete. Artifact execution starting.
2023-09-11 09:22:11 - Job Run IN_PROGRESS, Job run artifact execution in progress.
2023-09-11 09:21:58 - *****************************************
2023-09-11 09:21:58 - WARNING:torch.distributed.run:
2023-09-11 09:21:58 - *****************************************
2023-09-11 09:21:58 - Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the

In [ ]:
%%capture --no-stderr cap
job_run.watch()

with open('output1000_atco2.txt', 'w') as f:
    f.write(cap.stdout)